In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'


# Load the data
train_data = np.load('../Files/extracted_features/layer_features_lv60_self/train_0.npz')
test_data = np.load('../Files/extracted_features/layer_features_lv60_self/test_0.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = balanced_accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))


# # SVM Classification
# from sklearn.svm import SVC

# svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
# svm_model.fit(train_features, train_labels_encoded)
# svm_predictions = svm_model.predict(test_features)

# svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
# print(f"SVM Accuracy: {svm_accuracy:.4f}")
# print("SVM Classification Report:")
# print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
# ann_model = Sequential([
#     Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
#     Dropout(0.5),
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(len(label_encoder.classes_), activation='softmax')
# ])

# ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Train the ANN
# ann_history = ann_model.fit(
#     train_features_normalized, 
#     train_labels_encoded, 
#     epochs=50, 
#     batch_size=32, 
#     validation_split=0.2,
#     callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
# )

# # Evaluate the ANN
# ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
# print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# # ANN Predictions and Classification Report
# ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
# print("ANN Classification Report:")
# print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
# class CNN1DModel(nn.Module):
#     def __init__(self, input_channels, num_classes):
#         super(CNN1DModel, self).__init__()
#         self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
#         self.bn1 = nn.BatchNorm1d(32)
#         self.relu = nn.ReLU()
#         self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
#         self.bn2 = nn.BatchNorm1d(64)
#         self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
#         self.bn3 = nn.BatchNorm1d(128)
#         self.flatten = nn.Flatten()
#         self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
#         self.fc2 = nn.Linear(512, num_classes)
#         self.dropout = nn.Dropout(0.5)
    
#     def forward(self, x):
#         x = self.relu(self.bn1(self.conv1(x)))
#         x = self.relu(self.bn2(self.conv2(x)))
#         x = self.relu(self.bn3(self.conv3(x)))
#         x = self.flatten(x)
#         x = self.dropout(self.relu(self.fc1(x)))
#         x = self.fc2(x)
#         return x

# # Prepare tensors for PyTorch
# X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
# y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
# X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
# y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

# train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
# test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# # Initialize PyTorch model, loss, and optimizer
# input_channels = 1
# num_classes = len(label_encoder.classes_)
# model = CNN1DModel(input_channels, num_classes)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # PyTorch Training Loop
# epochs = 100
# for epoch in range(epochs):
#     model.train()
#     running_loss = 0.0
#     for inputs, labels in train_loader:
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item() * inputs.size(0)

#     epoch_loss = running_loss / len(train_loader.dataset)
#     print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# # Test PyTorch model
# def test_model(model, test_loader, criterion):
#     model.eval()
#     test_loss = 0.0
#     correct = 0
#     total = 0

#     with torch.no_grad():
#         for inputs, labels in test_loader:
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#             test_loss += loss.item() * inputs.size(0)
#             _, predicted = torch.max(outputs, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()

#     avg_loss = test_loss / total
#     accuracy = correct / total
#     return avg_loss, accuracy

# # Evaluate PyTorch model
# test_loss, test_accuracy = test_model(model, test_loader, criterion)
# print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
# base 100h

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, balanced_accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('../Files/extracted_features/layer_features_base_100/train_1.npz')
test_data = np.load('../Files/extracted_features/layer_features_base_100/test_1.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = balanced_accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))


In [ ]:
# base 960h

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, balanced_accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('../Files/extracted_features/layer_features_base_960/train_2.npz')
test_data = np.load('../Files/extracted_features/layer_features_base_960/test_2.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = balanced_accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))


In [ ]:
# hubert

In [ ]:
# 0
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_0.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_0.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 0, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 0, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 0, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")




In [ ]:
# 1
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_1.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_1.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 1, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 1, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 1, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")




In [ ]:
# 2
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_2.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_2.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 2, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 2, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 2, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")

In [ ]:
# 3
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_3.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_3.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 3, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 3, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 3, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")

In [ ]:
# 4
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_4.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_4.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 4, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 4, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 4, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")

In [ ]:
# 5
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_5.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_5.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 5, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 5, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 5, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")

In [ ]:
# 6
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_6.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_6.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 6, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 6, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 6, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")

In [ ]:
# 7
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_7.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_7.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 7, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 7, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 7, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")

In [ ]:
# 8
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_8.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_8.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 8, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 8, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 8, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")

In [ ]:
# 9
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_9.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_9.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 9, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 9, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 9, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")

In [ ]:
# 10
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_10.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_10.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 10, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 10, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 10, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")

In [ ]:
# 11
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_11.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_11.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 11, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 11, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 11, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")

In [ ]:
# 12
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_12.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_12.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 12, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 12, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 12, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")

In [ ]:
# 13
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_13.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_13.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 13, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 13, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 13, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")

In [ ]:
# 14
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_14.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_14.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 14, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 14, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 14, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")

In [ ]:
# 15
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_15.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_15.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 15, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 15, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 15, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")

In [ ]:
# 16
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_16.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_16.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 16, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 16, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 16, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")

In [ ]:
# 17
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_17.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_17.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 17, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 17, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 17, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")

In [ ]:
# 18
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_18.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_18.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 18, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 18, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 18, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")

In [ ]:
# 19
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_19.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_19.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 19, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 19, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 19, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")

In [ ]:
# 20
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_20.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_20.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 20, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 20, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 20, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")

In [ ]:
# 21
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_21.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_21.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 21, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 21, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 21, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")

In [ ]:
# 22
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_22.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_22.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 22, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 22, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 22, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")

In [ ]:
# 23
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_23.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_23.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 23, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 23, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 23, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")

In [ ]:
# 24
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
train_data = np.load('../Files/extracted_features/hubert_large/train_24.npz')
test_data = np.load('../Files/extracted_features/hubert_large/test_24.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# SVM Classification
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
svm_model.fit(train_features, train_labels_encoded)
svm_predictions = svm_model.predict(test_features)

svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("SVM Classification Report:")
print(classification_report(test_labels_encoded, svm_predictions))

# Function to save confusion matrix
def save_confusion_matrix(conf_matrix, labels, model_name, layer_number, folder_name):
    os.makedirs(folder_name, exist_ok=True)  # Ensure the folder exists
    plt.figure(figsize=(10, 8))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis', xticklabels=labels, yticklabels=labels)
    plt.title(f'{model_name} Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.tight_layout()
    save_path = os.path.join(folder_name, f'conf_matrix_layer_{layer_number}.png')
    plt.savefig(save_path)
    plt.close()
    print(f"Saved confusion matrix for {model_name}: {save_path}")

# SVM Confusion Matrix
svm_conf_matrix = confusion_matrix(test_labels_encoded, svm_predictions)
save_confusion_matrix(svm_conf_matrix, label_encoder.classes_, "SVM", 24, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/svm_conf_matrix")

# ANN Model
ann_model = Sequential([
    Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the ANN
ann_history = ann_model.fit(
    train_features_normalized, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the ANN
ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# ANN Predictions and Classification Report
ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
print("ANN Classification Report:")
print(classification_report(test_labels_encoded, ann_predictions))

# ANN Confusion Matrix
ann_conf_matrix = confusion_matrix(test_labels_encoded, ann_predictions)
save_confusion_matrix(ann_conf_matrix, label_encoder.classes_, "ANN", 24, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/ann_conf_matrix")

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))

# CNN Confusion Matrix
cnn_conf_matrix = confusion_matrix(test_labels_encoded, cnn_predictions)
save_confusion_matrix(cnn_conf_matrix, label_encoder.classes_, "CNN", 24, "../Files/confusion_matrix/balanced_accuracy_gender_hubert/cnn_conf_matrix")




In [ ]:
# wavlm